# Reaction–diffusion · Front-speed sweep walkthrough

This notebook sketches how `code/reaction_diffusion/rd_front_speed_sweep.py` organises Fisher–KPP parameter scans. We preview the configuration grid, reuse the single-case simulator to inspect a row, and show where the sweep writes its summary CSV.


In [ ]:
import sys
from pathlib import Path


def _ensure_repo_on_path():
    cwd = Path.cwd().resolve()
    candidates = [cwd, cwd.parent, cwd.parent.parent]
    for candidate in candidates:
        utils_py = candidate / 'notebooks' / 'utils.py'
        if utils_py.exists():
            sys.path.insert(0, str(candidate))
            return candidate
    return cwd

_ensure_repo_on_path()

from notebooks.utils import ensure_repo_path, preview_json, timestamp_slug

repo_root = ensure_repo_path()
print(f"Using repo root: {repo_root}")



In [ ]:
try:
    from code.reaction_diffusion import rd_front_speed_sweep as sweep
    from code.reaction_diffusion.rd_front_speed_experiment import run_sim
except ModuleNotFoundError:
    from notebooks.utils import ensure_repo_path as _vdm_ensure_repo_path
    _vdm_ensure_repo_path()
    import importlib
    importlib.invalidate_caches()
    from code.reaction_diffusion import rd_front_speed_sweep as sweep
    from code.reaction_diffusion.rd_front_speed_experiment import run_sim

from itertools import product
import numpy as np

scan = {"N": [128], "D": [0.5, 1.0], "r": [0.1, 0.25], "level": [0.1, 0.5]}
cases = list(product(scan["N"], scan["D"], scan["r"], scan["level"]))
print(f"Demo grid contains {len(cases)} cases")
print(cases[:4])

params = dict(N=128, L=120.0, D=1.0, r=0.25, T=25.0, cfl=0.2, seed=3, level=0.1, x0=-40.0, fit_frac=(0.6, 0.9), noise_amp=0.0)
sim = run_sim(
    params["N"], params["L"], params["D"], params["r"], params["T"],
    params["cfl"], params["seed"],
    level=params["level"], x0=params["x0"], fit_frac=params["fit_frac"], noise_amp=params["noise_amp"],
)
rel_err = abs(sim["c_abs"] - sim["c_th"]) / (abs(sim["c_th"]) + 1e-12)
print(preview_json({
    "c_theory": sim["c_th"],
    "c_meas": sim["c_abs"],
    "rel_err": rel_err,
    "r2": sim["r2"],
}))


In [ ]:
try:
    from code.common import io_paths
except ModuleNotFoundError:
    from notebooks.utils import ensure_repo_path as _vdm_ensure_repo_path
    _vdm_ensure_repo_path()
    import importlib
    importlib.invalidate_caches()
    from code.common import io_paths


slug = timestamp_slug("rd_front_speed_sweep_demo")
csv_path = io_paths.log_path("reaction_diffusion", slug).with_suffix(".csv")
print(f"Summary CSV would be written to: {csv_path}")


## Notebook checklist

* The sweep module shells out to `rd_front_speed_experiment.py`; exercising `run_sim` here keeps the turnaround quick.
* Adjust the grid dictionary to mirror the CLI flags before launching a full scan.
* The summary CSV lives under `logs/reaction_diffusion/` (timestamped slug) exactly like scripted runs.
* Use the CSV for plotting or downstream analytics without rerunning the sweep.
